![Partners](../images/partners.png)

# Preprocess ERA5 Data

This notebook processes historical ERA5 reanalysis data to create climatological reference values needed for renewable energy forecasting.

## 1. Overview

ERA5 is the fifth generation ECMWF reanalysis for the global climate and weather for the past 8 decades. 

**In this notebook, we will:**
1. Load pre-downloaded ERA5 daily sum data (2020-2025)
2. Compute day-of-year climatologies (365-day average values)



In [ ]:
from pathlib import Path
import shutil

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Copy the shared data to the local folder
src = Path("/data/shared/renewable-energy-data")
dst = Path("./data")

# Create destination folder if missing
dst.mkdir(parents=True, exist_ok=True)

# Copy everything inside src → dst
for item in src.iterdir():
    target = dst / item.name
    if item.is_dir():
        shutil.copytree(item, target, dirs_exist_ok=True)
    else:
        shutil.copy2(item, target)

print("Copied all data into ./data")

Data has been downloaded from the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) as daily sums of surface solar radiation downwards (ssrd) over East Africa. The resulting file can be found in the `data/ERA5/` folder.

**Data specifications:**

We used the following specifications for downloading ERA5 data:
- **Temporal coverage:** 2020-2025
- **Frequency:** Daily sums (from 6-hourly data)
- **Region:** East Africa  [22°N, 21.5°E, -12°S, 52°E]


## 2. Define Variables and File Paths

In [ ]:
# Variable name mapping
var_dict = {
    'ssrd': 'ssrd',   # Surface solar radiation downwards
}

print(f"Variables to process: {list(var_dict.keys())}")

In [ ]:
# Data directory containing the pre-downloaded ERA5 files
data_path = Path("data/ERA5")

filename_dict = {
    "surface_solar_radiation_downwards": "surface_solar_radiation_downwards_stream-oper_daily-sum.nc",
}


## 3. Compute Day-of-Year Climatology

A **climatology** represents the average conditions over multiple years. Here, we'll compute a **day-of-year climatology** - the average value for each day of the year (1-365) across all years in our dataset (2020-2025).

**Mathematical formulation:**

For each day of the year $d$ (where $d \in [1, 365]$), the climatology is computed as:

$$\text{Climatology}_d = \frac{1}{N_d} \sum_{i=1}^{N_d} X_{d,i}$$

Where:
- $X_{d,i}$ is the observed value on day-of-year $d$ in year $i$
- $N_d$ is the number of years with data for day $d$ (typically 5-6 years in our dataset: 2020-2025)

**Example:** The climatology for day 180 (≈ June 29) is the average of solar radiation values from June 29th across all years 2020, 2021, 2022, 2023, 2024, and 2025.



In [ ]:
def compute_climatology(dataset: xr.Dataset, var_name: str) -> xr.Dataset:
    """
    Compute day-of-year climatology for a given dataset.
    
    Args:
        dataset: xarray Dataset with valid_time dimension
        var_name: Name of the variable to process
        
    Returns:
        Dataset with climatology (averaged by day-of-year)
    """

    # Add day-of-year coordinate
    ds_with_doy = dataset.assign_coords(dayofyear=dataset.valid_time.dt.dayofyear)
    
    # Group by day-of-year and compute mean
    climatology = ds_with_doy.groupby('dayofyear').sum('valid_time')
    
    # skip day 366 (leap day)
    climatology = climatology.sel(dayofyear=slice(1, 365))

    return climatology


climatology_output_dir = Path("data/climatology")
climatology_output_dir.mkdir(parents=True, exist_ok=True)

for var_name, filename in filename_dict.items():

    output_file = climatology_output_dir / f"{var_name}_climatology.nc"
    if output_file.exists():
        print(f"Skipping {var_name}: output file already exists")
        ds = xr.open_dataset(output_file)
        continue

    file_path = data_path / filename
    print(f"\nProcessing: {var_name}")
    
    # Load dataset
    print(f"  Loading data from {filename}...")
    ds = xr.open_dataset(file_path)
    
    # Get the actual variable name from the dataset
    data_vars = list(ds.data_vars)
    if len(data_vars) == 1:
        actual_var = data_vars[0]
    else:
        # Try common variable names
        actual_var = data_vars[0]
    
    # Compute climatology
    print(f"  Computing climatology...")
    climatology = compute_climatology(ds, actual_var)
    
    # Save climatology
    
    print(f"  Saving to {output_file}...")
    climatology.to_netcdf(output_file)
    
    # Close datasets to free memory
    ds.close()



## 4. Visualize the Climatology

Let's visualize the climatological patterns for different days of the year.

In [ ]:
# Visualize climatology for a specific day (e.g., day 180 ≈ June 29)
day_to_plot = 180

fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ds['ssrd'].sel(dayofyear=day_to_plot).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='YlOrRd',
    cbar_kwargs={'label': 'Surface Solar Radiation (J/m²)'}
)
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black')
ax.coastlines(linewidth=0.8, color='black')
gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

ax.set_title(f'ERA5 Climatology: Solar Radiation - Day {day_to_plot}')
plt.tight_layout()
plt.show()

print(f"✓ Plotted climatology for day {day_to_plot}")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})

ds["ssrd"].var(dim="dayofyear").plot(ax=ax, 
                                    transform=ccrs.PlateCarree())
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black')
ax.coastlines(linewidth=0.8, color='black')
gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
ax.set_title(f'ERA5 Climatology: Variance in daily solar Radiation')
plt.tight_layout()
plt.show()

## 5. Analyze at a Specific Location

Let's extract and visualize the annual cycle of a variable at a specific location.

In [ ]:
# Select a location (e.g., Nairobi, Kenya: -1.28°N, 36.82°E)
location_name = "Nairobi, Kenya"
lat_point = -1.28
lon_point = 36.82

# Load the solar radiation climatology
clim_ssrd_reload = xr.open_dataset(climatology_output_dir / "surface_solar_radiation_downwards_climatology.nc")

# Extract time series at the location
location_series = clim_ssrd_reload['ssrd'].sel(
    latitude=lat_point, 
    longitude=lon_point, 
    method='nearest'
)

# Plot the annual cycle
fig, ax = plt.subplots(figsize=(14, 6))
location_series.plot(ax=ax, marker='o', markersize=3, linewidth=2, color='orangered')

ax.set_xlabel('Day of Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Surface Solar Radiation (J/m²)', fontsize=12, fontweight='bold')
ax.set_title(f'Annual Cycle of Solar Radiation - {location_name} ({lat_point}°N, {lon_point}°E)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add monthly markers
month_days = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax.set_xticks(month_days)
ax.set_xticklabels(month_names)

plt.tight_layout()
plt.show()

print(f"✓ Annual cycle plotted for {location_name}")
print(f"  Nearest grid point: {float(location_series.latitude):.2f}°N, {float(location_series.longitude):.2f}°E")
print(f"  Mean value: {float(location_series.mean()):.2f} J/m²")
print(f"  Max value: {float(location_series.max()):.2f} J/m² (day {int(location_series.argmax())})")
print(f"  Min value: {float(location_series.min()):.2f} J/m² (day {int(location_series.argmin())})")

In [ ]:
# show climatology of the next 14 days

from datetime import datetime, timedelta
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
print(f"Today's date: {today.strftime('%Y-%m-%d')}")

# Select a location (e.g., Nairobi, Kenya: -1.28°N, 36.82°E)
location_name = "Nairobi, Kenya"
lat_point = -1.28
lon_point = 36.82

# Load the solar radiation climatology
clim_ssrd_reload = xr.open_dataset(climatology_output_dir / "surface_solar_radiation_downwards_climatology.nc")

# Extract time series at the location
location_series = clim_ssrd_reload['ssrd'].sel(
    latitude=lat_point, 
    longitude=lon_point, 
    method='nearest'
).sel(dayofyear=slice(today.timetuple().tm_yday, today.timetuple().tm_yday + 14))

# Plot the annual cycle
fig, ax = plt.subplots(figsize=(14, 6))
location_series.plot(ax=ax, marker='o', markersize=3, linewidth=2, color='orangered')

ax.set_xlabel('Day of Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Surface Solar Radiation (J/m²)', fontsize=12, fontweight='bold')
ax.set_title(f'Climatology of Solar Radiation - 14 days ahead - {location_name} ({lat_point}°N, {lon_point}°E)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#### 8. Resources

- [ERA5 Documentation](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation)